Lien du projet kaggle :
https://www.kaggle.com/code/jyotiprasadpal/project-building-a-conversational-chatbot/notebook

In [62]:
import numpy as np
import pandas as pd
import torch
import os
import re
import time

from torchtext.vocab import Vocab
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as transforms
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn import Embedding, LSTM

from sklearn.model_selection import train_test_split

from collections import Counter

In [3]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
device = torch.device(device)

mps


In [4]:
lines = open('./chatbot_data/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conv_lines = open('./chatbot_data/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [5]:
lines[100].split(' +++$+++ ')[:-1]

['L394', 'u4', 'm0', 'JOEY']

In [6]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')

    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [7]:
id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [8]:
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    convs.append((_line.split((","))))

In [9]:
convs

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [10]:
questions = []
targets = []

for conv in convs:
    for i in range(len(conv) - 1):
        questions.append(id2line[conv[i]])
        targets.append(id2line[conv[i+1]])

print(len(questions))
print(len(targets))

221616
221616


In [11]:
def clean_text(text):
    text = text.lower()

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [12]:
clean_questions = []
clean_targets = []

for quest in questions:
    clean_questions.append(clean_text(quest))

for targ in targets:
    clean_targets.append(clean_text(targ))

In [13]:
clean_questions

['can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part please',
 'you are asking me out that is so cute that is your name again',
 'no no it is my fault we did not have a proper introduction',
 'cameron',
 'the thing is cameron i am at the mercy of a particularly hideous breed of loser my sister i cannot date until she does',
 'why',
 'unsolved mystery she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend',
 "c'esc ma tete this is my head",
 'right see you are ready for the quiz',
 'i do not want to know how to say that though i want to know useful things like where the good stores are how much does champagne cost stuff like chat i have never in my life had to point out my head to s

In [32]:
VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
MAX_LEN = 20
NUM_SAMPLES = 60000
GLOVE_DIR = './chatbot_data/glove-global/'
EPOCHS = 40
BATCH_SIZE = 256

In [15]:
for i, val in enumerate(zip(clean_questions, clean_targets)):
    print(i)
    print(val[0])
    print(val[1])
    break

0
can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again
well i thought we would start with pronunciation if that is okay with you


In [16]:
min_line_length = 2
max_line_length = 20

short_questions = []
short_answers = []

for i, val in enumerate(zip(clean_questions, clean_targets)):
    if (len(val[0].split()) >= min_line_length and len(val[0].split()) <= max_line_length ) and (
            min_line_length <= len(val[1].split()) <= max_line_length - 2):
        short_questions.append(val[0])
        short_answers.append(val[1])

print(len(short_questions))
print(len(short_answers))


133681
133681


In [17]:
input_text = short_questions[:NUM_SAMPLES]

In [18]:
def tagger(input):
    bos = "<BOS> "
    eos = " <EOS>"

    final_target = [bos + text + eos for text in input]
    return final_target

decoder_input_text = tagger(short_answers[:NUM_SAMPLES])

In [19]:
def vocab_creator(text_list, VOCAB_SIZE):

    def yield_tokens(text_list):
        for text in text_list:
            yield text.split()

    special_tokens = ["<pad>", "<unk>"]

    vocab = build_vocab_from_iterator(yield_tokens(text_list), specials=special_tokens, max_tokens=VOCAB_SIZE)

    word2idx = {word: idx for idx, word in enumerate(vocab.get_itos())}
    idx2word = {idx: word for idx, word in enumerate(vocab.get_itos())}

    return word2idx, idx2word


In [20]:
word2idx, idx2word = vocab_creator(text_list=input_text+decoder_input_text, VOCAB_SIZE=VOCAB_SIZE)

In [21]:
def text2seq(word2idx, idx2word, encoder_text, decoder_text):
    encoder_sequences = []
    decoder_sequences = []

    for text in encoder_text:
        subtext = []
        for word in text.split():
            if word in word2idx:
                subtext.append(word2idx[word])
            else:
                subtext.append(word2idx["<unk>"])
        encoder_sequences.append(subtext)

    for text in decoder_text:
        subtext = []
        for word in text.split():
            if word in word2idx:
                subtext.append(word2idx[word])
            else:
                subtext.append(word2idx["<unk>"])
        decoder_sequences.append(subtext)

    return encoder_sequences, decoder_sequences

In [22]:
encoder_sequences, decoder_sequences = text2seq(word2idx, idx2word, input_text, decoder_input_text)

In [23]:
def padding_sequence(encoder_sequences, decoder_sequences):
    encoder_tensor = [torch.tensor(sequence, dtype = torch.long) for sequence in encoder_sequences]
    decoder_tensor = [torch.tensor(sequence, dtype = torch.long) for sequence in decoder_sequences]

    encoder_padded = pad_sequence(encoder_tensor, batch_first=True)
    decoder_padded = pad_sequence(decoder_tensor, batch_first=True)

    return encoder_padded.tolist(), decoder_padded.tolist()

In [25]:
encoder_padded, decoder_padded = padding_sequence(encoder_sequences, decoder_sequences)

In [26]:
decoder_padded

[[2, 8, 6, 10333, 20, 10246, 20, 5888, 410, 140, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2,
  102,
  85,
  42,
  37,
  19,
  222,
  49,
  83,
  1066,
  19711,
  1782,
  151,
  3,
  0,
  0,
  0,
  0,
  0,
  0],
 [2, 290, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 122, 16, 70, 15, 5, 53, 12, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 55, 70, 4, 14, 341, 28, 6, 11039, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 290, 1066, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 52, 121, 255, 5, 51, 217, 32, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 4, 14, 658, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2,
  1,
  831,
  8132,
  275,
  130,
  353,
  20,
  5,
  96,
  150,
  278,
  11,
  18717,
  307,
  6,
  1,
  18377,
  3,
  0],
 [2,
  5,
  100,
  100,
  100,
  326,
  59,
  41,
  5,
  72,
  8,
  855,
  30,
  561,
  539,
  3,
  0,
  0,
  0,
  0],
 [2, 5, 22, 348, 34, 10, 41, 46, 97, 8, 502, 9, 32, 64, 28, 50, 3, 0, 0, 0],
 [2, 5, 146, 19, 1132, 84, 1136, 8526, 3, 0, 0, 0, 0, 0, 

#### Reshape the data to neural network shape

In [27]:
num_samples = len(encoder_padded)

In [28]:
def decoder_output_creator(decoder_input_data, num_samples, MAX_LEN, VOCAB_SIZE):
    decoder_output_data = np.zeros((num_samples, MAX_LEN, VOCAB_SIZE), dtype = "float32")

    for i, seqs in enumerate(decoder_input_data):
        for t, seq in enumerate(seqs):
            if t > 0:
                decoder_output_data[i][t][seq] = 1

    return decoder_output_data


In [29]:
decoder_output_data = decoder_output_creator(decoder_padded, num_samples, MAX_LEN, VOCAB_SIZE)

In [30]:
decoder_output_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [33]:
def glove_100d_dictionary(glove_dir):
    embedding_index = {}
    f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype = 'float32')
        embedding_index[word] = coefs
    f.close()
    return embedding_index

In [34]:
embedding_index = glove_100d_dictionary(GLOVE_DIR)

In [44]:
len(word2idx)

20000

In [45]:
def embedding_matrix_creator(max_word, embedding_dim):
    embedding_matrix = np.zeros((max_word, embedding_dim))

    for word, i in word2idx.items():
        embedding_vector = embedding_index.get(word)

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [46]:
embedding_matrix = embedding_matrix_creator(VOCAB_SIZE, EMBEDDING_DIM)

In [49]:
embedding_matrix.shape

(20000, 100)

In [51]:
def embedding_layer_creator(VOCAB_SIZE, EMBEDDING_DIM):
    embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)

    return embedding_layer

In [54]:
embedding_layer = embedding_layer_creator(VOCAB_SIZE, EMBEDDING_DIM)

In [57]:
embedding_layer.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype = torch.float32))
embedding_layer.weight.requires_grad = False


In [60]:
def data_spliter(encoder_padded, decoder_padded, test_size1, test_size2):
    en_train, en_test, de_train, de_test = train_test_split(encoder_padded, decoder_padded, test_size=test_size1)
    en_train, en_val, de_train, de_val = train_test_split(en_train, de_train, test_size = test_size2)

    return en_train, en_val, en_test, de_train, de_val, de_test

In [61]:
en_train, en_val, en_test, de_train, de_val, de_test = data_spliter(encoder_padded, decoder_padded, 0.2, 0.2)

In [ ]:
def build_seq2seq_model(HIDDEN_DIM=300):
    encder_inputs =

In [67]:
len(encoder_padded)

60000